# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tsiombe,-25.3000,45.4833,23.36,84,79,5.01,MG,1703025554
1,1,yellowknife,62.4560,-114.3525,-22.47,84,75,4.63,CA,1703025554
2,2,edinburgh of the seven seas,-37.0676,-12.3116,16.61,92,88,8.51,SH,1703025554
3,3,imbituba,-28.2400,-48.6703,23.61,89,20,5.54,BR,1703025406
4,4,ciudad pemex,17.9000,-92.5000,23.34,83,24,2.31,MX,1703025554


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display
# Configure the map plot
# YOUR CODE HERE
humidity = city_data_df["Humidity"].astype(int)
# Display the map
# YOUR CODE HERE# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df[
                            (city_data_df['Max Temp'] < 27 ) &
                            (city_data_df['Max Temp'] > 21 ) &
                            (city_data_df['Wind Speed'] < 4.5 ) & 
                            (city_data_df['Cloudiness'] == 0)
                            ]

# Drop any rows with null values
# YOUR CODE HERE
ideal_city_data_df = ideal_city_data_df.dropna()
# Display sample data
# YOUR CODE HERE
ideal_city_data_df 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
272,272,saint-pierre,-21.3393,55.4781,25.87,83,0,3.09,RE,1703025572
291,291,kassala,15.4510,36.4000,25.65,41,0,3.23,SD,1703025573
302,302,salalah,17.0151,54.0924,24.05,60,0,1.54,OM,1703025574
343,343,san miguel cuyutlan,20.4167,-103.4000,23.62,27,0,0.00,MX,1703025577
347,347,alice springs,-23.7000,133.8833,23.75,23,0,4.12,AU,1703025386
362,362,le vauclin,14.5452,-60.8388,26.03,69,0,3.09,MQ,1703025578
470,470,al ghayzah,16.2079,52.1760,22.80,67,0,2.41,YE,1703025585
516,516,basse-terre,16.1667,-61.6667,22.58,78,0,1.54,GP,1703025384
528,528,al hawatah,13.4167,34.6333,26.95,24,0,3.03,SD,1703025588
576,576,sao joao da barra,-21.6403,-41.0511,26.05,82,0,2.46,BR,1703025545


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_city_data_df.copy()
del hotel_df['City_ID']
del hotel_df['Max Temp']
del hotel_df['Cloudiness']
del hotel_df['Wind Speed']
del hotel_df['Date']
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
272,saint-pierre,-21.3393,55.4781,83,RE,
291,kassala,15.4510,36.4000,41,SD,
302,salalah,17.0151,54.0924,60,OM,
343,san miguel cuyutlan,20.4167,-103.4000,27,MX,
347,alice springs,-23.7000,133.8833,23,AU,
362,le vauclin,14.5452,-60.8388,69,MQ,
470,al ghayzah,16.2079,52.1760,67,YE,
516,basse-terre,16.1667,-61.6667,78,GP,
528,al hawatah,13.4167,34.6333,24,SD,
576,sao joao da barra,-21.6403,-41.0511,82,BR,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter": "",
    "bias": "",
    "categories": "accommodation.hotel",
    "distance": radius,  
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
kassala - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
san miguel cuyutlan - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
le vauclin - nearest hotel: Hôtel Macabou
al ghayzah - nearest hotel: فندق تاج العرب
basse-terre - nearest hotel: Le Créoliles
al hawatah - nearest hotel: No hotel found
sao joao da barra - nearest hotel: Pousada Mediterrâneo


,City,Lat,Lng,Humidity,Country,Hotel Name
272,saint-pierre,-21.3393,55.4781,83,RE,Tropic Hotel
291,kassala,15.4510,36.4000,41,SD,No hotel found
302,salalah,17.0151,54.0924,60,OM,Muscat International Hotel
343,san miguel cuyutlan,20.4167,-103.4000,27,MX,No hotel found
347,alice springs,-23.7000,133.8833,23,AU,Aurora Alice Springs
362,le vauclin,14.5452,-60.8388,69,MQ,Hôtel Macabou
470,al ghayzah,16.2079,52.1760,67,YE,فندق تاج العرب
516,basse-terre,16.1667,-61.6667,78,GP,Le Créoliles
528,al hawatah,13.4167,34.6333,24,SD,No hotel found
576,sao joao da barra,-21.6403,-41.0511,82,BR,Pousada Mediterrâneo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)
# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)